In [ ]:
# Import required libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv('/home/incent/conflixis-analytics/common/.env')

# Set up BigQuery client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
client = bigquery.Client(project='data-analytics-389803')

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load BigQuery magic commands
from IPython import get_ipython
if get_ipython() is not None:
    get_ipython().run_line_magic('load_ext', 'google.cloud.bigquery')
    print("✅ BigQuery magic commands loaded")

print("✅ Environment initialized successfully")
print(f"📊 Project: {client.project}")
print(f"🔑 Using credentials: {os.environ['GOOGLE_APPLICATION_CREDENTIALS']}")

# Risk Assessment Analysis - BCBS NPI Payment Data

## Overview
This notebook analyzes healthcare payment data from the BCBS NPI Risk Assessment dataset containing approximately 7.5 million payment records.

## Data Source
- **Project**: data-analytics-389803
- **Dataset**: Conflixis_sandbox
- **Main Table**: bcbs_npi_ra_v2
- **Secondary Table**: bcbs_ra_npi
- **Record Count**: 7,514,026 payments

## Key Columns
- `npi`: National Provider Identifier
- `specialty`: Medical specialty
- `entity_name`: Healthcare facility/hospital name
- `entity_id`: Unique entity identifier
- `payment`: Payment amount
- `payment_record`: Record identifier
- `nop`: Nature of payment (e.g., "Royalty or License", "Food and Beverage", "Consulting Fee")

## Analysis Sections
1. Data Overview and Exploration
2. Entity-Level Payment Analysis
3. Specialty-Level Payment Analysis
4. Nature of Payment (NOP) Analysis
5. Specific Specialty Deep Dives (Cardiology, Oncology, Surgery)

## Important Notes
- All aggregations are performed in BigQuery to minimize data transfer
- Only summarized results are downloaded locally
- Queries use LIMIT clauses only for exploration, not for complete analysis
- The `%%bigquery` magic command is loaded in the initialization cell

In [4]:
# Query configuration for optimal performance
job_config = bigquery.QueryJobConfig(
    use_query_cache=True,
    use_legacy_sql=False,
)

# Helper function to run queries safely
def run_query(query, job_config=job_config):
    """Run a BigQuery query and return results as a DataFrame"""
    try:
        query_job = client.query(query, job_config=job_config)
        results = query_job.to_dataframe()
        print(f"✅ Query completed. Rows returned: {len(results):,}")
        print(f"📊 Bytes processed: {query_job.total_bytes_processed:,}")
        return results
    except Exception as e:
        print(f"❌ Query failed: {str(e)}")
        return None

# Test connection
test_query = "SELECT 1 as test"
test_result = run_query(test_query)
if test_result is not None:
    print("🎉 BigQuery connection successful!")
else:
    print("⚠️ Please check your BigQuery credentials and connection")

✅ Query completed. Rows returned: 1
📊 Bytes processed: 0
🎉 BigQuery connection successful!


### Troubleshooting BigQuery Magic Commands

If the `%%bigquery` magic command doesn't work, try running this cell:

In [ ]:
# Alternative: Manually ensure BigQuery magic is loaded
try:
    get_ipython().magic('load_ext google.cloud.bigquery')
    print("✅ BigQuery magic extension loaded successfully!")
except Exception as e:
    print(f"❌ Error loading BigQuery magic: {e}")
    print("\nAlternative: Use the run_query() function instead of %%bigquery magic")
    
# Test the magic command
try:
    get_ipython().run_cell_magic('bigquery', '', 'SELECT 1 as test')
    print("✅ BigQuery magic command is working!")
except Exception as e:
    print(f"❌ BigQuery magic test failed: {e}")
    print("\nPlease use the run_query() function for queries instead.")

## 1. Data Overview and Exploration

### Sample NPI Lookup
Let's start by looking at a specific NPI to understand the data structure:

In [6]:
%%bigquery --location=us-east4
select * from  `data-analytics-389803.Conflixis_sandbox.bcbs_ra_npi`
where npi in (1033293196)
limit 10



UsageError: Cell magic `%%bigquery` not found.


### Explore Available Specialties
View a sample of distinct medical specialties in the dataset:

In [61]:
%%bigquery --location=us-east4
select distinct specialty
from data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2
limit 10


Query is running:   0%|          |

Downloading:   0%|          |

,specialty
0,Nurse - Nurse Practitioner
1,Family Practice
2,Endocrinology
3,General Practice
4,Ophthalmology
5,Internal Medicine
6,Physician Assistant
7,Psychiatry
8,Oncology - Hematology/Oncology
9,Obstetrics/Gynecology


### Total Record Count
Check the total number of payment records in the dataset:

In [ ]:
%%bigquery --location=us-east4
select count(payment_record) as total_record
from data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2

Query is running:   0%|          |

Downloading:   0%|          |

,total_record
0,7514026


## 2. Entity-Level Payment Analysis

### Top Entities by Payment Count
Identify healthcare entities with the highest number of payment records:

In [ ]:
%%bigquery --location=us-east4
select
entity_name,
count(payment_record) as op_payment,
from data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2
group by entity_name
order by op_payment desc
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,entity_name,op_payment
0,AdventHealth Orlando (FKA Florida Hospital Orl...,40258
1,Corewell Health William Beaumont University Ho...,38419
2,St Josephs Hospital,31427
3,Corewell Health Butterworth Hospital (FKA Spec...,29718
4,Huntsville Hospital,26533
5,Corewell Health Dearborn Hospital (AKA Beaumon...,25589
6,Piedmont Atlanta Hospital (AKA Piedmont Hospital),25295
7,Northside Hospital Atlanta,24765
8,Baptist Medical Center Jacksonville (FKA BMC D...,24063
9,Sarasota Memorial Hospital,23689


In [ ]:
# Visualize top entities by payment count
top_entities_count = pd.DataFrame({
    'entity_name': ['AdventHealth Orlando', 'Corewell Health William Beaumont', 'St Josephs Hospital', 
                    'Corewell Health Butterworth', 'Huntsville Hospital', 'Corewell Health Dearborn',
                    'Piedmont Atlanta Hospital', 'Northside Hospital Atlanta', 'Baptist Medical Center Jacksonville',
                    'Sarasota Memorial Hospital'],
    'op_payment': [40258, 38419, 31427, 29718, 26533, 25589, 25295, 24765, 24063, 23689]
})

# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(12, 8))
top_entities_count_sorted = top_entities_count.sort_values('op_payment')
bars = ax.barh(top_entities_count_sorted['entity_name'], top_entities_count_sorted['op_payment'])

# Customize the plot
ax.set_xlabel('Number of Payment Records', fontsize=12)
ax.set_title('Top 10 Healthcare Entities by Payment Count', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, (idx, row) in enumerate(top_entities_count_sorted.iterrows()):
    ax.text(row['op_payment'] + 500, i, f"{row['op_payment']:,}", 
            va='center', fontsize=10)

plt.tight_layout()
plt.show()

### Top Entities by Total Payment Value
Identify healthcare entities receiving the highest total payment amounts:

In [39]:
%%bigquery --location=us-east4
select
entity_name,
sum(payment) as payment_value,
from data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2
group by entity_name
order by payment_value desc
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,entity_name,payment_value
0,Mayo Clinic Hospital - Saint Marys Campus,26890436.58
1,Baptist Memorial Hospital - Memphis,19301946.17
2,Cedars-Sinai Medical Center,15484693.94
3,Cleveland Clinic Main Campus,9254037.74
4,Jefferson Bucks Hospital,8469980.20
5,Atrium Health Carolinas Medical Center,7890084.41
6,Corewell Health William Beaumont University Ho...,7288492.89
7,UAB St Vincents Birmingham (FKA Ascension St V...,6626692.45
8,Elmhurst Hospital,6538806.69
9,Baylor Scott & White Institute for Rehabilitat...,6325751.11


In [ ]:
# Visualize top entities by payment value
top_entities_value = pd.DataFrame({
    'entity_name': ['Mayo Clinic Hospital', 'Baptist Memorial Hospital', 'Cedars-Sinai Medical Center',
                    'Cleveland Clinic', 'Jefferson Bucks Hospital', 'Atrium Health Carolinas',
                    'Corewell Health William Beaumont', 'UAB St Vincents Birmingham', 'Elmhurst Hospital',
                    'Baylor Scott & White Institute'],
    'payment_value': [26890436.58, 19301946.17, 15484693.94, 9254037.74, 8469980.20,
                      7890084.41, 7288492.89, 6626692.45, 6538806.69, 6325751.11]
})

# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(12, 8))
top_entities_value_sorted = top_entities_value.sort_values('payment_value')
bars = ax.barh(top_entities_value_sorted['entity_name'], top_entities_value_sorted['payment_value'])

# Color bars by value
colors = plt.cm.viridis(top_entities_value_sorted['payment_value'] / top_entities_value_sorted['payment_value'].max())
for bar, color in zip(bars, colors):
    bar.set_color(color)

# Customize the plot
ax.set_xlabel('Total Payment Value ($)', fontsize=12)
ax.set_title('Top 10 Healthcare Entities by Total Payment Value', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, (idx, row) in enumerate(top_entities_value_sorted.iterrows()):
    ax.text(row['payment_value'] + 500000, i, f"${row['payment_value']/1e6:.1f}M", 
            va='center', fontsize=10)

plt.tight_layout()
plt.show()

### Payment Distribution by Nature of Payment (NOP)
Analyze how payments are distributed across different payment categories:

In [56]:
%%bigquery --location=us-east4
SELECT
entity_name, nop,
sum(payment) as payment_value
FROM `data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2`
GROUP BY entity_name,nop
ORDER BY payment_value DESC
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,entity_name,nop,payment_value
0,Mayo Clinic Hospital - Saint Marys Campus,Royalty or License,22246405.08
1,Baptist Memorial Hospital - Memphis,Royalty or License,17869823.90
2,Jefferson Bucks Hospital,Royalty or License,8459144.40
3,Cedars-Sinai Medical Center,Royalty or License,8347664.85
4,UAB St Vincents Birmingham (FKA Ascension St V...,Royalty or License,6067845.15
5,Atrium Health Carolinas Medical Center,Royalty or License,5912770.31
6,Elmhurst Hospital,Royalty or License,5539924.56
7,Cleveland Clinic Main Campus,Consulting Fee,4141388.21
8,NewYork-Presbyterian Columbia University Irvin...,Royalty or License,4043650.54
9,Cedars-Sinai Medical Center,Acquisitions,3644558.02


### Food and Beverage Payments Analysis
Specifically analyze entities receiving "Food and Beverage" payments:

In [46]:
%%bigquery --location=us-east4
SELECT
entity_name,
SUM(payment) AS payment_value,
count(payment_record) as record_id
FROM `data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2`
WHERE nop LIKE '%Food and Beverage%'
GROUP BY entity_name
ORDER BY payment_value DESC
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,entity_name,payment_value,record_id
0,AdventHealth Orlando (FKA Florida Hospital Orl...,1062955.98,36530
1,Corewell Health William Beaumont University Ho...,927339.13,34950
2,Corewell Health Butterworth Hospital (FKA Spec...,774573.11,26738
3,Cleveland Clinic Main Campus,765660.33,16134
4,St Josephs Hospital,715501.55,29945
5,Northside Hospital Atlanta,620637.89,22669
6,Piedmont Atlanta Hospital (AKA Piedmont Hospital),613256.48,23396
7,Sarasota Memorial Hospital,587856.90,21973
8,Corewell Health Dearborn Hospital (AKA Beaumon...,584389.88,24333
9,Huntsville Hospital,548093.81,25880


## 3. Specialty-Level Payment Analysis

### Top Specialties by Total Payment Value
Analyze which medical specialties receive the highest total payments:

In [57]:
%%bigquery --location=us-east4
SELECT
specialty,
SUM(payment) AS payment_value,
count(payment_record) as record_id
FROM `data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2`
GROUP BY specialty
ORDER BY payment_value DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,specialty,payment_value,record_id
0,Surgery - Orthopedic Surgery,2.356482e+08,147952
1,Dermatology,5.465900e+07,323834
2,Nurse - Nurse Practitioner,4.987298e+07,1448777
3,Neurology,4.427430e+07,322464
4,Cardiology - Cardiologist,3.559282e+07,284381
5,Surgery - Neurosurgery,3.315164e+07,20354
6,Internal Medicine,3.284878e+07,685349
7,Family Practice,2.914465e+07,926929
8,Gastroenterology,2.896145e+07,312346
9,Physician Assistant,2.746087e+07,763226


In [ ]:
# Visualize specialty payment distribution
specialty_data = pd.DataFrame({
    'specialty': ['Surgery - Orthopedic Surgery', 'Dermatology', 'Nurse - Nurse Practitioner',
                  'Neurology', 'Cardiology - Cardiologist', 'Surgery - Neurosurgery',
                  'Internal Medicine', 'Family Practice', 'Gastroenterology', 'Physician Assistant'],
    'payment_value': [235648200, 54659000, 49872980, 44274300, 35592820, 33151640,
                      32848780, 29144650, 28961450, 27460870],
    'record_count': [147952, 323834, 1448777, 322464, 284381, 20354,
                     685349, 926929, 312346, 763226]
})

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Plot 1: Payment values
specialty_value_sorted = specialty_data.sort_values('payment_value', ascending=True)
bars1 = ax1.barh(specialty_value_sorted['specialty'], specialty_value_sorted['payment_value'])
ax1.set_xlabel('Total Payment Value ($)', fontsize=12)
ax1.set_title('Top Specialties by Payment Value', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(specialty_value_sorted.iterrows()):
    ax1.text(row['payment_value'] + 2000000, i, f"${row['payment_value']/1e6:.1f}M", 
            va='center', fontsize=9)

# Plot 2: Record counts
specialty_count_sorted = specialty_data.sort_values('record_count', ascending=True)
bars2 = ax2.barh(specialty_count_sorted['specialty'], specialty_count_sorted['record_count'])
ax2.set_xlabel('Number of Payment Records', fontsize=12)
ax2.set_title('Top Specialties by Record Count', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(specialty_count_sorted.iterrows()):
    ax2.text(row['record_count'] + 20000, i, f"{row['record_count']:,}", 
            va='center', fontsize=9)

plt.tight_layout()
plt.show()

# Calculate and display average payment per record
specialty_data['avg_payment'] = specialty_data['payment_value'] / specialty_data['record_count']
avg_payment_sorted = specialty_data.sort_values('avg_payment', ascending=False)

print("\nAverage Payment per Record by Specialty:")
print("-" * 50)
for _, row in avg_payment_sorted.iterrows():
    print(f"{row['specialty']:.<40} ${row['avg_payment']:,.2f}")

### Orthopedic Specialties
Explore orthopedic-related specialties in the dataset:

In [65]:

%%bigquery --location=us-east4
select distinct specialty
from data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2
where specialty like '%Ortho%'
limit 10



Query is running:   0%|          |

Downloading:   0%|          |

,specialty
0,Surgery - Orthopedic Surgery
1,Surgery - Orthopedic Spine Surgery
2,Surgery - Pediatric Orthopedic Surgery


## 4. Specialty-Specific Deep Dives

### Cardiology Payment Analysis
Top entities receiving payments for cardiology services:

In [59]:
%%bigquery --location=us-east4
SELECT
entity_name,
SUM(payment) AS payment_value,
count(payment_record) as record_id
FROM `data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2`
WHERE specialty LIKE '%Cardiology%'
GROUP BY entity_name
ORDER BY payment_value DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,entity_name,payment_value,record_id
0,University of California Davis Medical Center ...,1113034.72,514
1,Cleveland Clinic Main Campus,1082267.63,2561
2,University Hospital,779908.37,1827
3,Henry Ford Hospital,713217.23,1220
4,UNC Rex Hospital (FKA Rex Hospital),686985.22,667
5,Piedmont Atlanta Hospital (AKA Piedmont Hospital),670646.90,1948
6,Cedars-Sinai Medical Center,612109.33,3046
7,OhioHealth Riverside Methodist Hospital,603965.51,1398
8,AdventHealth Orlando (FKA Florida Hospital Orl...,598260.38,2645
9,Ascension St John Hospital,596966.73,1492


### Oncology Payment Analysis
Top entities receiving payments for oncology services:

In [63]:
%%bigquery --location=us-east4
SELECT
entity_name,
SUM(payment) AS payment_value,
count(payment_record) as record_id
FROM `data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2`
WHERE specialty LIKE '%Oncology%'
GROUP BY entity_name
ORDER BY payment_value DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,entity_name,payment_value,record_id
0,AdventHealth Orlando (FKA Florida Hospital Orl...,1642468.00,2528
1,Baylor University Medical Center - Dallas,1234138.80,1888
2,University of Texas MD Anderson Cancer Center,1228485.47,1502
3,Cleveland Clinic Main Campus,960602.59,954
4,Helford Clinical Research Hospital (AKA City o...,899858.97,916
5,Moffitt Cancer Center Magnolia Campus (AKA H L...,746968.92,803
6,University Hospital,717863.57,610
7,Froedtert Hospital,628056.48,388
8,Baylor Scott & White Institute for Rehabilitat...,560710.22,612
9,Ascension St Vincent Hospital - Indianapolis,558918.57,888


### Surgery Payment Analysis
Top entities receiving payments for surgical services:

In [66]:
%%bigquery --location=us-east4
SELECT
entity_name,
SUM(payment) AS payment_value,
count(payment_record) as record_id
FROM `data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2`
WHERE specialty LIKE '%Surgery%'
GROUP BY entity_name
ORDER BY payment_value DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,entity_name,payment_value,record_id
0,Mayo Clinic Hospital - Saint Marys Campus,23779083.11,2105
1,Baptist Memorial Hospital - Memphis,18122053.04,738
2,Jefferson Bucks Hospital,8459218.89,8
3,Atrium Health Carolinas Medical Center,6448116.57,1202
4,UAB St Vincents Birmingham (FKA Ascension St V...,6309373.63,679
5,Elmhurst Hospital,6201619.48,546
6,Cedars-Sinai Medical Center,4642341.79,879
7,Baylor Scott & White Institute for Rehabilitat...,4310947.67,2435
8,NewYork-Presbyterian Columbia University Irvin...,4277365.17,203
9,Vail Health (FKA Vail Valley Medical Center),3956508.82,345


## 5. Summary Statistics and Key Findings

In [ ]:
# Calculate and display summary statistics
print("📊 BCBS NPI Risk Assessment - Summary Statistics")
print("=" * 60)
print(f"\n📈 Dataset Overview:")
print(f"  • Total Payment Records: 7,514,026")
print(f"  • Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}")
print(f"  • Data Source: BigQuery - data-analytics-389803.Conflixis_sandbox")

print(f"\n🏥 Top Healthcare Entities:")
print(f"  • By Payment Count: AdventHealth Orlando (40,258 payments)")
print(f"  • By Payment Value: Mayo Clinic Hospital ($26.9M)")

print(f"\n💊 Top Medical Specialties:")
print(f"  • By Payment Value: Surgery - Orthopedic Surgery ($235.6M)")
print(f"  • By Record Count: Nurse - Nurse Practitioner (1,448,777 records)")
print(f"  • Highest Avg Payment: Surgery - Neurosurgery ($1,628.66/record)")

print(f"\n💰 Payment Categories (Nature of Payment):")
print(f"  • Dominant Category: Royalty or License payments")
print(f"  • Mayo Clinic's Royalty payments alone: $22.2M")

print(f"\n🍽️ Food and Beverage Payments:")
print(f"  • Top Recipient: AdventHealth Orlando ($1.06M, 36,530 records)")
print(f"  • Total F&B Records (top 10 entities): ~270,000")

print(f"\n🔬 Specialty-Specific Insights:")
print(f"  • Cardiology: UC Davis Medical Center leads ($1.11M)")
print(f"  • Oncology: AdventHealth Orlando leads ($1.64M)")
print(f"  • Surgery: Mayo Clinic dominates ($23.8M)")

print("\n" + "=" * 60)

## 6. Export Results for Further Analysis

In [ ]:
# Create output directory if it doesn't exist
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

# Prepare summary data for export
summary_data = {
    'Top Entities by Payment Count': top_entities_count,
    'Top Entities by Payment Value': top_entities_value,
    'Top Specialties': specialty_data
}

# Export to Excel with multiple sheets
with pd.ExcelWriter(f'{output_dir}/bcbs_risk_assessment_summary.xlsx', engine='openpyxl') as writer:
    for sheet_name, df in summary_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Export individual CSV files
top_entities_count.to_csv(f'{output_dir}/top_entities_by_count.csv', index=False)
top_entities_value.to_csv(f'{output_dir}/top_entities_by_value.csv', index=False)
specialty_data.to_csv(f'{output_dir}/specialty_analysis.csv', index=False)

# Create a summary report
report_content = f"""BCBS NPI Risk Assessment - Executive Summary
Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

Dataset Overview:
- Total Records: 7,514,026
- Data Source: Google BigQuery
- Table: data-analytics-389803.Conflixis_sandbox.bcbs_npi_ra_v2

Key Findings:

1. Top Healthcare Entities by Payment Volume:
   - Mayo Clinic Hospital: $26.9M
   - Baptist Memorial Hospital: $19.3M
   - Cedars-Sinai Medical Center: $15.5M

2. Top Specialties by Total Payments:
   - Surgery - Orthopedic Surgery: $235.6M (147,952 records)
   - Dermatology: $54.7M (323,834 records)
   - Nurse - Nurse Practitioner: $49.9M (1,448,777 records)

3. Payment Categories:
   - "Royalty or License" payments dominate the highest value transactions
   - Food and Beverage payments are widespread across major hospitals

4. Notable Patterns:
   - Mayo Clinic receives exceptionally high royalty payments ($22.2M)
   - Nurse Practitioners have the highest record count but lower average payments
   - Orthopedic Surgery has the highest total payment value despite fewer records

Recommendations for Further Analysis:
- Investigate the nature of high-value royalty payments
- Analyze temporal trends if date data is available
- Compare payment patterns across geographic regions
- Examine correlations between entity size and payment types
"""

# Save the report
with open(f'{output_dir}/executive_summary.txt', 'w') as f:
    f.write(report_content)

print("✅ Results exported successfully!")
print(f"📁 Output directory: {output_dir}/")
print("\nExported files:")
print("  • bcbs_risk_assessment_summary.xlsx (multi-sheet workbook)")
print("  • top_entities_by_count.csv")
print("  • top_entities_by_value.csv")
print("  • specialty_analysis.csv")
print("  • executive_summary.txt")